In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data = pd.read_excel('/content/LabeledText.xlsx')

# y = data['LABEL']

drop_neutral = data[ data['LABEL'] == 'neutral' ].index
data.drop(drop_neutral, inplace = True)
data.loc[data["LABEL"] == "negative", "LABEL"] = 0
data.loc[data["LABEL"] == "positive", "LABEL"] = 1
data = data.astype({"LABEL": int})

caption = data['Caption']

y = data['LABEL']
print(y.count)
data

<bound method Series.count of 0       0
1       0
2       1
4       1
5       1
       ..
4862    0
4863    0
4864    1
4865    1
4868    1
Name: LABEL, Length: 3098, dtype: int64>


,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,0
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,0
2,100.txt,This is my Valentine's from 1 of my nephews. I...,1
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,1
5,1002.txt,Chaotic Love - giclee print ?65 at #art #love ...,1
...,...,...,...
4862,993.txt,#Fanghorn forest or #Alder #woodland up Glen #...,0
4863,994.txt,"Where there¡¯s a whisk, there¡¯s a way. #power...",0
4864,995.txt,OMG. Well done #Eskom! 'Man dies during #LoadS...,1
4865,996.txt,Feelin' the love in here! #ValentinesDay #caring,1


In [ ]:
import re

train_data = []
example_caption = ['Today is a great day!!', 
                   'I had a very bad day.',
                   'Please do not make me sad.',
                   'Be careful, they are coming.',
                   'He loves playing games! and you?',
                   'Had a very good meal at dinner :(',
                   'I love you Istanbul, you are a very beautiful city.',
                   "I'm so confused, what can I do about it? @muazfurkan",
                   'What are the recommendations for going abroad?',
                   'Days of shit await us.']
example_data = []

def cleaner(caption):
  pattern = r'[^\w\s]|_'

  for text in caption:
    text = re.sub(pattern, '', text)
    text = text.strip().split(' ')
    for i in range(len(text)):
      if text[i].startswith('http'):
        del text[i]
        break
    train_data.append(text)
  


def cleaner2(caption):
  pattern = r'[^\w\s]|_'
  for text in caption:
    text = re.sub(pattern, '', text)
    text = text.strip().split(' ')
    example_data.append(text)
  print(example_data)

if __name__ == "__main__":
   cleaner(caption)
   cleaner2(example_caption)

[['Today', 'is', 'a', 'great', 'day'], ['I', 'had', 'a', 'very', 'bad', 'day'], ['Please', 'do', 'not', 'make', 'me', 'sad'], ['Be', 'careful', 'they', 'are', 'coming'], ['He', 'loves', 'playing', 'games', 'and', 'you'], ['Had', 'a', 'very', 'good', 'meal', 'at', 'dinner'], ['I', 'love', 'you', 'Istanbul', 'you', 'are', 'a', 'very', 'beautiful', 'city'], ['Im', 'so', 'confused', 'what', 'can', 'I', 'do', 'about', 'it', 'muazfurkan'], ['What', 'are', 'the', 'recommendations', 'for', 'going', 'abroad'], ['Days', 'of', 'shit', 'await', 'us']]


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


In [ ]:
from gensim.models import Word2Vec

def vectorize(data):
  sentence_vectors = []
  for sentence in data:
    sentence_vector = np.array([])
    for word in sentence:
      try:
        word_vector = model[word]
        sentence_vector = np.concatenate((sentence_vector, word_vector))
      except KeyError:
        print('Word not in vocabulary: {}'.format(word))
    sentence_vectors.append(sentence_vector)
  return sentence_vectors


vectors = vectorize(train_data)
print(len(vectors))

In [ ]:
padded_vectors = []
lengths = []
pad_length = []

for vec in vectors:
  lengths.append(len(vec))

max_len = max(lengths)
vectors = np.array(vectors)
padded_vectors = np.zeros((len(vectors), max_len))

def padding(vectors):
  for i, vec in enumerate(vectors):
      padded_vectors[i, :vec.shape[0]] = vec
  return padded_vectors

vectors = padding(vectors)

<ipython-input-21-c4433ec4e2e9>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectors = np.array(vectors)


In [ ]:
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(vectors, y)):

  X_train, X_test = vectors[train_index], vectors[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  rf = RandomForestClassifier(n_estimators = 10)
  rf.fit(X_train, y_train)
  y_pred = rf.predict(X_test)
  pred_prob = rf.predict_proba(X_test)[:, 1]

  f1 = f1_score(y_test, y_pred)
  roc_auc = roc_auc_score(y_test, pred_prob)

  print('f1 : ', f1)
  print('roc-auc : ', roc_auc)

f1 :  0.6329113924050632
roc-auc :  0.6793929346012679
f1 :  0.6440677966101694
roc-auc :  0.6645205279813459
f1 :  0.652694610778443
roc-auc :  0.6744169042918768
f1 :  0.6234177215189873
roc-auc :  0.6602854122621564
f1 :  0.6275752773375596
roc-auc :  0.6696854196854196


In [ ]:
example_vectors = vectorize(example_data)
example_vectors = padding(example_vectors)

Word not in vocabulary: Today
Word not in vocabulary: I
Word not in vocabulary: Please
Word not in vocabulary: Be
Word not in vocabulary: He
Word not in vocabulary: Had
Word not in vocabulary: I
Word not in vocabulary: Istanbul
Word not in vocabulary: Im
Word not in vocabulary: I
Word not in vocabulary: muazfurkan
Word not in vocabulary: What
Word not in vocabulary: Days


In [ ]:
new_pred = rf.predict(example_vectors)